In [1]:
#libraries
from ortools.linear_solver import pywraplp as glp

In [2]:
#imputs
name=['frame','support','strap']
department = ['cutting','milling','shaping']
unit_component = [1.0,2.0,1.0] #One product is assembled from four components: 1 frame, 2 supports, 1 metal strap, 1 motor
#but we can neglect the motor info because the company does not control make/buy decision of it
#When we minimize the production cost of the other 3 components, we also minimize the production costs of making liftmaster
num_prod = 5000.0 #5000 liftmaster is demanded
cost = [[38.0,51.0],[11.5,15.0],[6.5,7.5]] #manufacturing vs purchase costs for frame, support, strap
time = [[3.5,2.2,3.1],[1.3,1.7,2.6],[0.8,0.0,1.7]] #time for each component in each department
capacity = [350.0,420.0,680.0]

In [3]:
#intermediate calculations
component = [(num_prod*i) for i in unit_component]
capacity_minute = [(i*60) for i in capacity]

In [4]:
##Create model
model = glp.Solver('Frandec',glp.Solver.GLOP_LINEAR_PROGRAMMING)

In [5]:
#Decision variables:
dvar = list()
#We arrange make quantities before buy quantities. 
#Within each category, we arranged according to the order of component in name list
for i in range(len(component)*2):
    if i < len(component):
        dvar.append(model.NumVar(0,model.infinity(),name[i]+' manufactured'))
    else:
        dvar.append(model.NumVar(0,model.infinity(), name[i-len(component)]+' purchased'))

In [6]:
#Objective Function
TotCost = model.Objective()
TotCost.SetMinimization()
for i in range(len(dvar)):
    if i < len(component):
        TotCost.SetCoefficient(dvar[i],cost[i][0])
    else:
        TotCost.SetCoefficient(dvar[i],cost[i-len(component)][1])

In [7]:
#Constraints
constr = list(range(len(component)+len(department)))
#3 constraints on number of components, total of make and buy for each component = number required to make 5000 lifemaster
for c in range(len(component)):
    constr[c]=model.Constraint(component[c],component[c],'Number of %s:'% name[c])
    for d in range(len(dvar)):
        if ((d==c) or (d==(c+len(component)))):
            constr[c].SetCoefficient(dvar[d],1)
        else:
            constr[c].SetCoefficient(dvar[d],0)
#3 constraints on department capacity
for i in range(len(component),len(component)+len(department)):
    constr[i]=model.Constraint(0,capacity_minute[i-len(component)],'Capacity of %s:'% department[i-len(component)])
    for d in range(len(dvar)):
        if (d<len(component)):
            constr[i].SetCoefficient(dvar[d],time[d][i-len(component)])
        else:
            constr[i].SetCoefficient(dvar[d],0)

In [8]:
#Solve model and display result
statusls = ['Optimal','Feasible','Infeasible','Unbounded','Abnormal','Not Solved']
status = model.Solve()
print('Solution status:', statusls[status])
print('Number of variables:', model.NumVariables())
print('Number of constraints:',model.NumConstraints())
for d in range(len(dvar)):
    if (d<len(component)):
        print('%s manufactured = %.0f units' % (name[d],dvar[d].solution_value()))
    else:
        print('%s purchased = %.0f units' % (name[d-len(component)],dvar[d].solution_value()))
print('Total production cost: $%.2f'%(TotCost.Value()))

Solution status: Optimal
Number of variables: 6
Number of constraints: 6
frame manufactured = 5000 units
support manufactured = 2692 units
strap manufactured = 0 units
frame purchased = 0 units
support purchased = 7308 units
strap purchased = 5000 units
Total production cost: $368076.92


In [9]:
#Display Constraints info
LHS = model.ComputeConstraintActivities()
for i in range(3):
    print('Total number of %s (both make and buy): %.0f, marginal cost: %.2f' % (name[i],LHS[i],constr[i].dual_value()))
for i in range(3):
    print('Department %s time (hours): %.2f, capacity: %.2f, marginal cost: %.2f' % (department[i],LHS[i+len(component)]/60,capacity[i],constr[i+len(component)].dual_value()))

Total number of frame (both make and buy): 5000, marginal cost: 47.42
Total number of support (both make and buy): 10000, marginal cost: 15.00
Total number of strap (both make and buy): 5000, marginal cost: 7.50
Department cutting time (hours): 350.00, capacity: 350.00, marginal cost: -2.69
Department milling time (hours): 259.62, capacity: 420.00, marginal cost: 0.00
Department shaping time (hours): 375.00, capacity: 680.00, marginal cost: 0.00


b) Frantec should pay for additional hour of each department based on the respective marginal cost shown above

In [10]:
#c
#new cost matrix
cost = [[38.0,45.0],[11.5,15.0],[6.5,7.5]] #manufacturing vs purchase costs for frame, support, strap
for i in range(len(dvar)):
    if i < len(component):
        TotCost.SetCoefficient(dvar[i],cost[i][0])
    else:
        TotCost.SetCoefficient(dvar[i],cost[i-len(component)][1])
status = model.Solve()
print('Solution status:', statusls[status])
for d in range(len(dvar)):
    if (d<len(component)):
        print('%s manufactured = %.0f units' % (name[d],dvar[d].solution_value()))
    else:
        print('%s purchased = %.0f units' % (name[d-len(component)],dvar[d].solution_value()))
print('Total production cost: $%.2f'%(TotCost.Value()))

Solution status: Optimal
frame manufactured = 2286 units
support manufactured = 10000 units
strap manufactured = 0 units
frame purchased = 2714 units
support purchased = 0 units
strap purchased = 5000 units
Total production cost: $361500.00


Since the new cost is lower than the old cost, the company should accept the deal